# Visualizing changes to the North Atlantic subtropical high (NASH)

Plot the mean 850-hPa geopotential height field during May-July for time periods 1979-2001 and 2002-2024. 

In [1]:
# Package imports
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import cmaps as ncl_cm

from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.ticker as mticker


## Retrieve the seasonal Z850 and UV925 fields


In [6]:
# Retrieve indices from data folder
tna_sst_early = pd.read_csv('../data/seasonal_indices/tna_MJJ_sst_index.csv', header='infer').set_index('time').to_xarray()['tna_sst']
tna_wind_early = pd.read_csv('../data/seasonal_indices/tna_MJJ_u925_index.csv', header='infer').set_index('time').to_xarray()['tna_u925']
sst_thresh = pd.read_csv('../data/seasonal_indices/sst_critical_MJJ_index.csv', header='infer').set_index('time').to_xarray()['sst_thresh']
cllj_early = pd.read_csv('../data/seasonal_indices/cllj_MJJ_index.csv', header='infer').set_index('time').to_xarray()['cllj_index']
tna_prcp_early = pd.read_csv('../data/seasonal_indices/tna_MJJ_prcp_index.csv', header='infer').set_index('time').to_xarray()['tna_prcp']
tna_rsst_early = pd.read_csv('../data/seasonal_indices/tna_MJJ_rsst_index.csv', header='infer').set_index('time').to_xarray()['tna_rsst']

# Convert time index to datetime64
tna_sst_early['time'] = pd.to_datetime(tna_sst_early['time'].values)
tna_wind_early['time'] = pd.to_datetime(tna_wind_early['time'].values)
sst_thresh['time'] = pd.to_datetime(sst_thresh['time'].values)
cllj_early['time'] = pd.to_datetime(cllj_early['time'].values)
tna_prcp_early['time'] = pd.to_datetime(tna_prcp_early['time'].values)
tna_rsst_early['time'] = pd.to_datetime(tna_rsst_early['time'].values)

## Plot the seasonal maps for each time period and the difference

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid # Import AxesGrid

fig = plt.figure(figsize=(10, 10), dpi=300) # Adjust figure size as needed

# Define subplots using AxesGrid
grid = AxesGrid(fig, 111, nrows_ncols=(3, 1),
                axes_class=axes_class,
                share_all=True,
                cbar_location='right',
                cbar_mode='edge', # Set to 'single' for a single colorbar
                cbar_pad=0.1,
                cbar_size='5%',
                axes_pad=0.4, # Padding between subplots
                direction='row')


# Define extent
extent = [-100, -20, 5, 35]
levels_z850 = np.arange(1520, 1605, 5)
levels_z850_diff = np.arange(-10, 11, 1)
subplot_titles = ['1979-2001', '2002-2024', 'Difference']
subplot_labels = ['a', 'b', 'c']

# Basic plotting loop (replace with your actual plotting data)
for i, ax in enumerate(grid):
    ax.set_extent(extent)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=2, color='black')
    ax.grid(visible=True, lw=0.5, ls=':', color='lightgray')
    # Modify gridlines to plot labels for all subplots
    # ax.gridlines(draw_labels=["bottom", "left"], linewidth=0.5, color='lightgray', alpha=0.5, linestyle='--')
    ax.set_xticks(np.arange(-90, -20, 10), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(5, 45, 5), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.tick_params(axis='both', which='major', labelsize=8)

    if i in [0,1]:
      # Add a placeholder plot (replace with your actual data plotting)
      im1 = ax.contourf(z850_period1.longitude,
                        z850_period1.latitude,
                        all_z850_data[i,:,:],
                        levels = levels_z850,
                        cmap=ncl_cm.WhiteBlueGreenYellowRed,
                        extend='both',
                        transform=ccrs.PlateCarree())
      cntr = ax.contour(z850_period1.longitude,
                        z850_period1.latitude,
                        all_z850_data[i,:,:],
                        levels=[1560],
                        colors='black',
                        linewidths=2,
                        linestyles='--',
                        transform=ccrs.PlateCarree())
      cntr.clabel(fmt='  %1.0f  ', fontsize=9)

      qv = ax.quiver(u925_period1.longitude,
                     u925_period1.latitude,
                     all_u925_data[i,:,:],
                     all_v925_data[i,:,:],
                     alpha=0.6,
                     units='inches',
                     scale=60,
                     transform=ccrs.PlateCarree())
      ax.quiverkey(qv, 0.85, 1.04, 10, r'10 m s$^{-1}$', labelpos='E',
                  coordinates='axes')

    else:
      # Add a placeholder plot (replace with your actual data plotting)
      im2 = ax.contourf(z850_period1.longitude,
                        z850_period1.latitude,
                        all_z850_data[i,:,:],
                        levels = levels_z850_diff,
                        cmap=ncl_cm.temp_diff_18lev,
                        extend='both',
                        transform=ccrs.PlateCarree())

      qv = ax.quiver(u925_period1.longitude,
                      u925_period1.latitude,
                      all_u925_data[i,:,:],
                      all_v925_data[i,:,:],
                      alpha=0.6,
                      units='inches',
                      scale=8,
                      transform=ccrs.PlateCarree())
      ax.quiverkey(qv, 0.87, 1.04, 2, r'2 m s$^{-1}$', labelpos='E',
                  coordinates='axes')

    # Add a title and label
    ax.set_title(subplot_titles[i], loc='left', fontsize=12, fontweight='bold')
    ax.text(0.02, 0.98, subplot_labels[i], transform=ax.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')

# Add a single colorbar for the grid (if cbar_mode is 'single')
cbar = grid.cbar_axes[0].colorbar(im1)
cbar.ax.tick_params(labelsize=8)
cbar.set_label('(m)', fontsize=8)

cbar = grid.cbar_axes[1].colorbar(im1)
cbar.ax.tick_params(labelsize=8)
cbar.set_label('(m)', fontsize=8)

cbar = grid.cbar_axes[2].colorbar(im2)
cbar.ax.tick_params(labelsize=8)
cbar.set_label('(m)', fontsize=8)


plt.show()

# ---- Save figure ----
fig.savefig('../figures/figure4.pdf', dpi=300, format='pdf', bbox_inches='tight')